In [ ]:
%pip -q install langchain openai langchain_openai TA-Lib-Precompiled

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.3/291.3 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.2/115.2 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 36.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import tool
import yfinance as yf
from langchain.prompts import MessagesPlaceholder
from langchain_core.messages import AIMessage, HumanMessage
from langchain.agents.format_scratchpad.openai_tools import (
format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
from langchain.agents import AgentExecutor


In [ ]:
from datetime import datetime, timedelta
import pandas as pd
import yfinance as yf
import talib

In [ ]:
@tool
def get_company_symbol(symbol:str) -> str:
  """Returns the ticker of the company inputted"""
  ticker = yf.Ticker(symbol)
  return ticker

In [ ]:
@tool
def get_stock_price(symbol:str) -> float:
  """Returns current price of ticker"""
  ticker = yf.Ticker(symbol)
  todays_data = ticker.history(period='1d')
  return round(todays_data['Close'][0], 2)

In [ ]:
import numpy as np

In [ ]:
@tool
def calculate_rsi(symbol:str) -> float:
    """Return RSI Calculation of ticker"""
    end_date = datetime.today().strftime('%Y-%m-%d')
    start_date = (datetime.today() - timedelta(days=100)).strftime('%Y-%m-%d')
    data = yf.download(symbol, start=start_date, end=end_date)
    delta = data['Adj Close'].diff(1)
    delta = delta[1:]
    up = delta.copy()
    down = delta.copy()
    up[up < 0] = 0
    down[down > 0] = 0
    avg_gain = up.rolling(window=14).mean()
    avg_loss = abs(down.rolling(window=14).mean())
    rs = avg_gain / avg_loss
    rsi = 100.0 - (100.0 / (1.0 + rs))
    data['RSI'] = rsi
    temp = data['RSI'].iloc[-1]
    rsi = format(temp,".2f")
    return rsi

In [ ]:
@tool
def ma(ticker:str) -> str:
    """Returns Moving Average of ticker"""
    # Define the function to calculate WMA
    def calculate_wma(prices):
        weights = np.arange(1, 15)
        wma = np.convolve(prices, weights, mode='valid') / weights.sum()
        return np.concatenate((np.full(13, np.nan), wma), axis=0)

    # Download stock data
    end_date = datetime.today().strftime('%Y-%m-%d')
    start_date = (datetime.today() - timedelta(days=100)).strftime('%Y-%m-%d')
    data = yf.download(ticker, start=start_date, end=end_date)

    if data.empty:
        return None  # Return None if no data is retrieved

    # Calculate SMA
    data['SMA'] = data['Close'].rolling(window=14).mean()

    # Calculate EMA
    data['EMA'] = data['Close'].ewm(span=14, adjust=False).mean()

    # Calculate WMA
    data['WMA'] = calculate_wma(data['Close'].values)
    mac = format(data['Close'].iloc[-1],".2f")
    mas = format(data['SMA'].iloc[-1],".2f")
    mae = format(data['EMA'].iloc[-1],".2f")
    maw = format(data['WMA'].iloc[-1],".2f")
    if (mac < mas):
      ta1 = f"Closing Value is less than SMA, It may indicate a downtrend. Closing Value : {mac} , SMA : {mas}. "
    else:
      ta1 = f"Closing Value is greater than SMA, It may indicate a uptrend. Closing Value : {mac} , SMA : {mas}. "
    if (mac < mas):
      ta2 = f"Closing Value is less than EMA, It may indicate a downtrend. Closing Value : {mac} , EMA : {mae}. "
    else:
      ta2 = f"Closing Value is greater than EMA, It may indicate a uptrend. Closing Value : {mac} , EMA : {mae}. "
    if (mac < mas):
      ta3 = f"Closing Value is less than WMA, It may indicate a downtrend. Closing Value : {mac} , WMA : {maw}. "
    else:
      ta3 = f"Closing Value is greater than WMA, It may indicate a uptrend. Closing Value : {mac} , WMA : {maw}. "

    x = ta1 + "\n" + ta2 + "\n" + ta3 + "\n"
    return x

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [ ]:
@tool
def predict_stock(ticker:str) -> float:
  """Predicts the next day's closing value using ticker"""

  # Fetch historical stock data from Yahoo Finance
  stock_data = yf.download(ticker,period='max')

  # Calculate additional features
  stock_data['SMA_50'] = stock_data['Close'].rolling(window=50).mean()  # 50-day Simple Moving Average
  stock_data['SMA_200'] = stock_data['Close'].rolling(window=200).mean()  # 200-day Simple Moving Average
  stock_data['Daily_Return'] = stock_data['Close'].pct_change()  # Daily Return
  stock_data['Volatility'] = stock_data['Daily_Return'].rolling(window=20).std()  # 20-day Volatility

  # Drop rows with NaN values
  stock_data.dropna(inplace=True)

  # Define features and target variable
  X=stock_data.drop(['Adj Close'] ,axis=1)# Features
  y = stock_data['Adj Close']
  regressor = RandomForestRegressor(n_estimators=100, max_depth=None)
  regressor.fit(X,y)
  X_latest = X.tail(1)
  # Make predictions using the latest feature set
  predictions = regressor.predict(X_latest)
  # Calculate MSE for each prediction
  mse_values = mean_squared_error(y.tail(1), predictions)

  # Get the best prediction
  best_prediction = predictions[0]
  predicted_price = format(best_prediction,".2f")
  # Print the best prediction
  return best_prediction

In [ ]:
@tool
def candlestick(ticker:str) -> str:
  """ReturnReturns the candlestick pattern as well as its learning by ticker"""
  candlestick_patterns = {
    'CDL2CROWS':'Two Crows',
    'CDL3BLACKCROWS':'Three Black Crows',
    'CDL3INSIDE':'Three Inside Up/Down',
    'CDL3LINESTRIKE':'Three-Line Strike',
    'CDL3OUTSIDE':'Three Outside Up/Down',
    'CDL3STARSINSOUTH':'Three Stars In The South',
    'CDL3WHITESOLDIERS':'Three Advancing White Soldiers',
    'CDLABANDONEDBABY':'Abandoned Baby',
    'CDLADVANCEBLOCK':'Advance Block',
    'CDLBELTHOLD':'Belt-hold',
    'CDLBREAKAWAY':'Breakaway',
    'CDLCLOSINGMARUBOZU':'Closing Marubozu',
    'CDLCONCEALBABYSWALL':'Concealing Baby Swallow',
    'CDLCOUNTERATTACK':'Counterattack',
    'CDLDARKCLOUDCOVER':'Dark Cloud Cover',
    'CDLDOJI':'Doji',
    'CDLDOJISTAR':'Doji Star',
    'CDLDRAGONFLYDOJI':'Dragonfly Doji',
    'CDLENGULFING':'Engulfing Pattern',
    'CDLEVENINGDOJISTAR':'Evening Doji Star',
    'CDLEVENINGSTAR':'Evening Star',
    'CDLGAPSIDESIDEWHITE':'Up/Down-gap side-by-side white lines',
    'CDLGRAVESTONEDOJI':'Gravestone Doji',
    'CDLHAMMER':'Hammer',
    'CDLHANGINGMAN':'Hanging Man',
    'CDLHARAMI':'Harami Pattern',
    'CDLHARAMICROSS':'Harami Cross Pattern',
    'CDLHIGHWAVE':'High-Wave Candle',
    'CDLHIKKAKE':'Hikkake Pattern',
    'CDLHIKKAKEMOD':'Modified Hikkake Pattern',
    'CDLHOMINGPIGEON':'Homing Pigeon',
    'CDLIDENTICAL3CROWS':'Identical Three Crows',
    'CDLINNECK':'In-Neck Pattern',
    'CDLINVERTEDHAMMER':'Inverted Hammer',
    'CDLKICKING':'Kicking',
    'CDLKICKINGBYLENGTH':'Kicking - bull/bear determined by the longer marubozu',
    'CDLLADDERBOTTOM':'Ladder Bottom',
    'CDLLONGLEGGEDDOJI':'Long Legged Doji',
    'CDLLONGLINE':'Long Line Candle',
    'CDLMARUBOZU':'Marubozu',
    'CDLMATCHINGLOW':'Matching Low',
    'CDLMATHOLD':'Mat Hold',
    'CDLMORNINGDOJISTAR':'Morning Doji Star',
    'CDLMORNINGSTAR':'Morning Star',
    'CDLONNECK':'On-Neck Pattern',
    'CDLPIERCING':'Piercing Pattern',
    'CDLRICKSHAWMAN':'Rickshaw Man',
    'CDLRISEFALL3METHODS':'Rising/Falling Three Methods',
    'CDLSEPARATINGLINES':'Separating Lines',
    'CDLSHOOTINGSTAR':'Shooting Star',
    'CDLSHORTLINE':'Short Line Candle',
    'CDLSPINNINGTOP':'Spinning Top',
    'CDLSTALLEDPATTERN':'Stalled Pattern',
    'CDLSTICKSANDWICH':'Stick Sandwich',
    'CDLTAKURI':'Takuri (Dragonfly Doji with very long lower shadow)',
    'CDLTASUKIGAP':'Tasuki Gap',
    'CDLTHRUSTING':'Thrusting Pattern',
    'CDLTRISTAR':'Tristar Pattern',
    'CDLUNIQUE3RIVER':'Unique 3 River',
    'CDLUPSIDEGAP2CROWS':'Upside Gap Two Crows',
    'CDLXSIDEGAP3METHODS':'Upside/Downside Gap Three Methods'
  }
  start_date = datetime.today() - timedelta(days=100)
  df = yf.download(ticker, start=start_date, end=datetime.today())
  df = df.reset_index()
  df=df.drop(columns=['Adj Close','Volume'])
  for pattern, pattern_name in candlestick_patterns.items():
      pattern_result = getattr(talib, pattern)(df['Open'], df['High'], df['Low'], df['Close'])
      df[pattern_name] = pattern_result.astype(bool)
  data=df
  data['PositiveEngulfing'] = data['Engulfing Pattern'] > 0
  data['NegativeEngulfing'] = data['Engulfing Pattern'] < 0
  data.drop(columns=['Engulfing Pattern'])
  pattern_trend = {
    'Two Crows': 'bearish',
    'Three Black Crows': 'bearish',
    'Three Inside Up/Down': 'bullish/bearish',
    'Three-Line Strike': 'bullish/bearish',
    'Three Outside Up/Down': 'bullish/bearish',
    'Three Stars In The South': 'bullish',
    'Three Advancing White Soldiers': 'bullish',
    'Abandoned Baby': 'bullish',
    'Advance Block': 'bearish',
    'Belt-hold': 'bullish',
    'Breakaway': 'bullish',
    'Closing Marubozu': 'bullish',
    'Concealing Baby Swallow': 'bullish',
    'Counterattack': 'bullish',
    'Dark Cloud Cover': 'bearish',
    'Doji': 'neutral',
    'Doji Star': 'neutral',
    'Dragonfly Doji': 'bullish',
    'Engulfing Pattern': 'bullish',
    'Evening Doji Star': 'bearish',
    'Evening Star': 'bearish',
    'Up/Down-gap side-by-side white lines': 'bullish',
    'Gravestone Doji': 'bearish',
    'Hammer': 'bullish',
    'Hanging Man': 'bearish',
    'Harami Pattern': 'bullish',
    'Harami Cross Pattern': 'bullish',
    'High-Wave Candle': 'neutral',
    'Hikkake Pattern': 'neutral',
    'Modified Hikkake Pattern': 'neutral',
    'Homing Pigeon': 'bullish',
    'Identical Three Crows': 'bearish',
    'In-Neck Pattern': 'bearish',
    'Inverted Hammer': 'bullish',
    'Kicking': 'bullish',
    'Kicking - bull/bear determined by the longer marubozu': 'bullish',
    'Ladder Bottom': 'bullish',
    'Long Legged Doji': 'neutral',
    'Long Line Candle': 'neutral',
    'Marubozu': 'bullish',
    'Matching Low': 'bullish',
    'Mat Hold': 'bullish',
    'Morning Doji Star': 'bullish',
    'Morning Star': 'bullish',
    'On-Neck Pattern': 'bearish',
    'Piercing Pattern': 'bullish',
    'Rickshaw Man': 'neutral',
    'Rising/Falling Three Methods': 'bullish',
    'Separating Lines': 'bullish',
    'Shooting Star': 'bearish',
    'Short Line Candle': 'neutral',
    'Spinning Top': 'neutral',
    'Stalled Pattern': 'neutral',
    'Stick Sandwich': 'neutral',
    'Takuri (Dragonfly Doji with very long lower shadow)': 'bullish',
    'Tasuki Gap': 'neutral',
    'Thrusting Pattern': 'bullish',
    'Tristar Pattern': 'neutral',
    'Unique 3 River': 'bullish',
    'Upside Gap Two Crows': 'bearish',
    'Upside/Downside Gap Three Methods': 'bullish',
    'PositiveEngulfing': 'bullish',
    'NegativeEngulfing': 'bearish'
  }
  pattern_columns = data.columns[5:]

  latest_dates = {}

  for column in pattern_columns:
      pattern_data = data[data[column]]
      if not pattern_data.empty:
          latest_date = pattern_data['Date'].iloc[-1]
          latest_dates[column] = latest_date

  Candle_DF = pd.DataFrame(list(latest_dates.items()), columns=['Date','Pattern'])
  Candle_DF_sorted = Candle_DF.sort_values(by='Date', ascending=True)
  latest_row = Candle_DF_sorted.loc[2]
  latest_date = latest_row['Date']
  latest_pattern = latest_row['Pattern']
  x=f'Latest Date: {latest_pattern} \nPattern: {latest_date} \nTrend: {pattern_trend[latest_date]}'
  return x

In [ ]:
tools = [get_company_symbol,get_stock_price,calculate_rsi,ma,predict_stock,candlestick]

In [ ]:
from langchain_community.llms import HuggingFaceEndpoint
from langchain_community.chat_models.huggingface import ChatHuggingFace

In [ ]:
llm = HuggingFaceEndpoint(huggingfacehub_api_token= "api_token",repo_id="mistralai/Mixtral-8x7B-Instruct-v0.1")

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
chat_model = ChatHuggingFace(llm=llm)

In [ ]:
tools = [get_company_symbol,get_stock_price,calculate_rsi,ma,predict_stock,candlestick]

In [ ]:
from langchain.agents import initialize_agent
agent = initialize_agent(tools,
                         llm,
                         agent="zero-shot-react-description",
                         verbose = True,
                         handle_parsing_errors=True)

In [ ]:
agent.agent.llm_chain.prompt.template

"Answer the following questions as best you can. You have access to the following tools:\n\nget_company_symbol: get_company_symbol(symbol: str) -> str - Returns the ticker of the company inputted\nget_stock_price: get_stock_price(symbol: str) -> float - Returns current price of ticker\ncalculate_rsi: calculate_rsi(symbol: str) -> float - Return RSI Calculation of ticker\nma: ma(ticker: str) -> str - Returns Moving Average of ticker\npredict_stock: predict_stock(ticker: str) -> float - Predicts the next day's closing value using ticker\ncandlestick: candlestick(ticker: str) -> str - ReturnReturns the candlestick pattern as well as its learning by ticker\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [get_company_symbol, get_stock_price, calculate_rsi, ma, predict_stock, candlestick]\nAction Input: the input to the action\nObservation: the result of the action\n.

In [ ]:
sys_message = "You are StockAI, a stock market assistant. Answer the following questions as best you can. You have access to the following tools:\n\nget_company_symbol: get_company_symbol(symbol: str) -> str - Returns the ticker of the company inputted - Use this before all other tools get_stock_price: get_stock_price(symbol: str) -> float - Returns current price of ticker calculate_rsi: calculate_rsi(symbol: str) -> float - Return RSI Calculation of ticker ma: ma(ticker: str) -> str - Returns Moving Average of ticker predict_stock: predict_stock(ticker: str) -> float - Predicts the next day's closing value using ticker candlestick: candlestick(ticker: str) -> str - Returns the candlestick pattern as well as its learning by ticker\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [get_company_symbol, get_stock_price, calculate_rsi, ma, predict_stock, candlestick]\nAction Input: the input to the action\nObservation: the result of the action... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: show outputs of all observations and answer to the input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}"

In [ ]:
agent.agent.llm_chain.prompt.template = sys_message

In [ ]:
agent.agent.llm_chain.prompt.template

"You are StockAI, a stock market assistant. Answer the following questions as best you can. You have access to the following tools:\n\nget_company_symbol: get_company_symbol(symbol: str) -> str - Returns the ticker of the company inputted - Use this before all other tools get_stock_price: get_stock_price(symbol: str) -> float - Returns current price of ticker calculate_rsi: calculate_rsi(symbol: str) -> float - Return RSI Calculation of ticker ma: ma(ticker: str) -> str - Returns Moving Average of ticker predict_stock: predict_stock(ticker: str) -> float - Predicts the next day's closing value using ticker candlestick: candlestick(ticker: str) -> str - Returns the candlestick pattern as well as its learning by ticker\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [get_company_symbol, get_stock_price, calculate_rsi, ma, predict_stock, candlestick]\nAction Input:

In [ ]:
agent.invoke("is goog doing well")



> Entering new AgentExecutor chain...
 I first need to get the stock symbol for the company named goog
Action: get_company_symbol
Action Input: goog
Observation: yfinance.Ticker object <GOOG>
Thought: Now that I have the stock symbol, I can get the current price of the stock to determine if it is doing well
Action: get_stock_price
Action Input: GOOG
Observation: 155.72
Thought:

[*********************100%%**********************]  1 of 1 completed

 The current price is 155.72. I should check the RSI to see if the stock is overbought or oversold.
Action: calculate_rsi
Action Input: GOOG
Observation: 48.33
Thought:

 The RSI is 48.33, this indicates that the stock is neither overbought nor oversold.
Thought: I now know the final answer
Final Answer: Based on the current stock price of 155.72 and an RSI of 48.33, GOOG is doing well.

> Finished chain.


{'input': 'is goog doing well',
 'output': 'Based on the current stock price of 155.72 and an RSI of 48.33, GOOG is doing well.'}

In [ ]:
agent.invoke("should i invest in NVIDIA")



> Entering new AgentExecutor chain...
 I need to gather information about NVIDIA's stock performance and predict its future trend. I will use the get_company_symbol tool to get NVIDIA's symbol, then use get_stock_price, calculate_rsi, ma, and predict_stock to analyze its stock performance and trend.
Action: get_company_symbol
Action Input: NVIDIA
Observation: yfinance.Ticker object <NVIDIA>
Thought: Now I have NVIDIA's symbol, I will check its current price using get_stock_price.
Action: get_stock_price
Action Input: NVDA
Observation: 762.0
Thought:

[*********************100%%**********************]  1 of 1 completed

 Now I know NVIDIA's current price, I will calculate its RSI to understand its volatility.
Action: calculate_rsi
Action Input: NVDA
Observation: 28.536366892260872
Thought:


[*********************100%%**********************]  1 of 1 completed

 Now I have calculated NVIDIA's RSI, I will check its moving average to understand its trend.
Action: ma
Action Input: NVDA
Observation: Closing Value is less than SMA, It may indicate a downtrend. Closing Value : 762.0 , SMA : 863.5564313616071. 
Closing Value is less than EMA, It may indicate a downtrend. Closing Value : 762.0 , EMA : 856.7045104627579. 
Closing Value is less than WMA, It may indicate a downtrend. Closing Value : 762.0 , WMA : 874.7024803524926. 

Thought:

 Now I have checked NVIDIA's RSI and moving average, I will predict its next day's closing value using predict_stock.
Action: predict_stock
Action Input: NVDA

[*********************100%%**********************]  1 of 1 completed



Observation: 780.6218139648438
Thought: I now know the final answer
Final Answer: Based on the analysis, NVIDIA's current price is $762.0 with a RSI of 28.536366892260872, indicating low volatility. Its moving average suggests a downtrend. However, the prediction tool shows an increase in price to $780.6218139648438 for the next day. Therefore, investing in NVIDIA could be a good decision considering the prediction but it is always recommended to do further research before making any investment decisions.

> Finished chain.


{'input': 'should i invest in NVIDIA',
 'output': "Based on the analysis, NVIDIA's current price is $762.0 with a RSI of 28.536366892260872, indicating low volatility. Its moving average suggests a downtrend. However, the prediction tool shows an increase in price to $780.6218139648438 for the next day. Therefore, investing in NVIDIA could be a good decision considering the prediction but it is always recommended to do further research before making any investment decisions."}

In [ ]:
reponse = agent.invoke("Should I invest in wells fargo")



> Entering new AgentExecutor chain...
 To answer this question, I should first find out the current price of Wells Fargo stock, then calculate the RSI (Relative Strength Index) to check the stock's stability, and finally predict the stock's price for tomorrow.
Action: get_company_symbol
Action Input: Wells Fargo
Observation: yfinance.Ticker object <WELLS FARGO>
Thought: Now I know the symbol for Wells Fargo is WELLS FARGO or WFC. I will use WFC for future actions.
Action: get_stock_price
Action Input: WFC
Observation: 60.35
Thought: Now I know the current price of Wells Fargo stock is 60.35 dollars. Next, I will calculate the RSI.
Action: calculate_rsi
Action Input: WFC

[*********************100%%**********************]  1 of 1 completed



Observation: 66.39
Thought: Now I know the RSI for Wells Fargo is 66.39. This means the stock is overbought and may be due for a correction. However, I still need to predict the stock's price for tomorrow to make a final decision.
Action: predict_stock
Action Input: WFC

[*********************100%%**********************]  1 of 1 completed



Observation: 58.838435134887696
Thought: Now I know the predicted price of Wells Fargo for tomorrow is 58.84 dollars. Considering the high RSI and the predicted price drop, it might not be the best idea to invest in Wells Fargo right now.
Final Answer: The current price of Wells Fargo stock is 60.35 dollars, the RSI is 66.39, and the predicted price for tomorrow is 58.84 dollars. Due to the high RSI and the predicted price drop, it might not be the best idea to invest in Wells Fargo right now.

> Finished chain.


In [ ]:
print(reponse['output'])

The current price of Wells Fargo stock is 60.35 dollars, the RSI is 66.39, and the predicted price for tomorrow is 58.84 dollars. Due to the high RSI and the predicted price drop, it might not be the best idea to invest in Wells Fargo right now.


In [ ]:
agent.invoke("should i invest in NVIDIA")